In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
df=df.drop(columns=['id'],axis=1)

In [ ]:
# Independent and Dependent features
X=df.drop(labels=['price'],axis=1)
Y=df['price']

In [8]:
# Segregating dataset with categorical and numerical variables so that categorical variables can be encoded
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [9]:
# define ranking for each ordinal/categorical variable
cut_categories=['Fair','Good','Very Good','Premium','Ideal']
color_categories=['D','E','F','G','H','I','J']
clarity_categories=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
!pip install scikit-learn
from sklearn.impute import SimpleImputer  # handling missing values
from sklearn.preprocessing import StandardScaler  # scaling numerical variables
from sklearn.preprocessing import OrdinalEncoder  # encoding categorical variables
# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------  9.2/9.3 MB 43.9 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 38.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
   ----------- ---------------------------- 11.8/42.2 MB 61.7 MB/s eta 0:00:01
   ----------------------- ---------------- 24.9/42.2 MB 60.7 MB/s eta 0:00:01
   ----------------------------------- ---- 37.0/42.2 MB 60.3 MB/s eta 0:00:01
   ---------------------------------------  42.2/42.2 MB 53.7 MB/s eta 0:00:01
   ---------------------------------------- 42.2/42.2 MB 48.8 MB/s eta 0:00:00


In [17]:
# Numerical Pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),  # handling missing values
        ('scaler',StandardScaler())  # scaling numerical variables

    ]
)

# Categorical Pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy="most_frequent")),  # handling missing values
        ('encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),  # encoding categorical variables
        ('scaler',StandardScaler())  # scaling variables
    ]
)

preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)  # applying categorical pipeline to categorical variables
]
)

In [13]:
# Train test split
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [18]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [19]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,-0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,-0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,1.526720,-0.648127


In [ ]:
# Model Training
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [23]:
# Model evaluation function
import numpy as np
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
def evaluate_model(y_true,y_pred):
    r2=r2_score(y_true,y_pred)
    mae=mean_absolute_error(y_true,y_pred)
    mse=mean_squared_error(y_true,y_pred)
    rmse=np.sqrt(mse)
    return r2,mae,mse,rmse

In [24]:
## Train multiple models
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.linear_model import LinearRegression

models={
    'Linear Regression':LinearRegression(),
    'Lasso Regression':Lasso(),
    'Ridge Regression':Ridge(),
    'ElasticNet Regression':ElasticNet()

}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

# Make predictions for each model
    Y_pred=model.predict(X_test)
    r2,mae,mse,rmse=evaluate_model(Y_test,Y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('R2 Score:',r2*100)
    print('Mean Absolute Error:',mae)
    print('Mean Squared Error:',mse)
    print('Root Mean Squared Error:',rmse)

    r2_list.append(r2)

    print('='*35)
    print('\n')


Linear Regression
Model Training Performance
R2 Score: 93.62906819996049
Mean Absolute Error: 675.0758270067483
Mean Squared Error: 1029473.3531156846
Root Mean Squared Error: 1014.6296630375463


Lasso Regression
Model Training Performance
R2 Score: 93.62869814082755
Mean Absolute Error: 676.2421173665508
Mean Squared Error: 1029533.1506505491
Root Mean Squared Error: 1014.659130275064


Ridge Regression
Model Training Performance
R2 Score: 93.62900967491632
Mean Absolute Error: 675.1077629781329
Mean Squared Error: 1029482.8101268952
Root Mean Squared Error: 1014.6343233534411


ElasticNet Regression
Model Training Performance
R2 Score: 85.44967219374031
Mean Absolute Error: 1060.9432977143008
Mean Squared Error: 2351174.8713978743
Root Mean Squared Error: 1533.3541245902313




In [25]:
model_list

['Linear Regression',
 'Lasso Regression',
 'Ridge Regression',
 'ElasticNet Regression']